# Imports

In [1]:
import sys
import random
import numpy as np
from io import StringIO

from gym import Env, spaces

# Variables

Here you can define the size of the board (size x size), the number of connections you have to have to win and the number of players

In [ ]:
size = 4
connect = 3
aggents = 3

Here is the object Board, This object contains all the rules of the game

In [ ]:
class Board():
    
    def __init__(self, size = 4, connect = 3, aggents = 2):
        self.shape = (size, size)
        
        
        self.table = np.zeros(self.shape)

        self.size = size
        self.connect = connect
        self.aggents = aggents
    
    
    def reset(self):
        self.table = np.zeros(self.shape)
        
    def render(self):
        output = ''

        # i is row
        # j is columns
        for i in range(self.size):
            for j in range(self.size):
                output = output + ' ' + str(int(self.table[i,j]))

            print(output)
            output = ''
            
    def step(self, player, action):
        for i in range(self.size):
            try:
                if self.table[i+1][action] != 0:
                    self.table[i][action] = player
                    return self.checkWinner()
            except:
                self.table[i][action] = player
                return self.checkWinner()
        print('Error occurred')

    def checkIfMoveAvailable(self, action):
        if self.table[0][action] != 0:
            return False
        return True
    
    def numberOfPlayers(self):
        return self.aggents
    
    def returnSize(self):
        return self.shape
    
    def returnTable(self):
        return self.table
            
    def checkWinner(self):

        for player in range(1, self.aggents + 1, 1):
            
            # check columns
            for i in range(self.size):
                for j in range(self.size):
                    try:
                        num = 0
                        while(num >= 0):
                            if self.table[i + num][j] == player:
                                num = num + 1
                            else:
                                num = -1
                            if num >= self.connect:
                                return player
                    except:
                        pass

            # check rows
            for i in range(self.size):
                for j in range(self.size):
                    try:
                        num = 0
                        while(num >= 0):
                            if self.table[i][j + num] == player:
                                num = num + 1
                            else:
                                num = -1
                            if num >= self.connect:
                                return player
                    except:
                        pass
                        
            #check diagonals
            for i in range(self.size):
                for j in range(self.size):
                    try:
                        num = 0
                        while(num >= 0):
                            if self.table[i + num][j + num] == player:
                                num = num + 1
                            else:
                                num = -1
                            if num >= self.connect:
                                return player
                    except:
                        pass

                    try:
                        num = 0
                        while(num >= 0):
                            if self.table[i + num][j - num] == player:
                                num = num + 1
                            else:
                                num = -1
                            if num >= self.connect:
                                return player
                    except:
                        pass
            
        # game is not finished yet
        for i in range(self.size):
            for j in range(self.size):
                if self.table[i][j] == 0:
                    return 0
                    
        # they tie
        return -1
        
        
    

Just a method for helping me to render the table

In [ ]:
def render(table):
    output = ''

    # i is row
    # j is columns
    for i in range(table.shape[0]):
        for j in range(table.shape[0]):
            output = output + ' ' + str(int(table[i,j]))

        print(output)
        output = ''

Creating the object

In [ ]:
env = Board(size, connect, aggents)

A method to check if the state is on the list or not

In [ ]:
def checkStateIsOnListOrAdd(states, table):
    for i in range(len(states)):
        if np.array_equal(states[i],table):
            return states, False, i
    tab = np.array(table.copy())
    states.append(tab.copy())
    return states, True, len(states)-1

# Main part

Because we have a huge space we are going to save every state as we came accros it

Our main player is player one, the other players are just selecting randomly the move

Player one use a q_table to select the best move

In [ ]:
q_table = []
states = []
    
# Hyperparameters
learning_rate = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_frames = []
epochs, penalties, victories = 0, 0, 0

for i in range(1, 2000): # 100001
    env.reset()

    winner = 0
        
    while winner == 0:
            
        reward = 0
            
        for player in range(env.numberOfPlayers()):
                
            actionAvailable = False
                
            states, new, statePosition = checkStateIsOnListOrAdd(states, env.returnTable())
                
            if new:
                q_table.append(np.zeros(size))
                
            if new == False and player == 1:
                action = np.argmax(q_table[statePosition])
            else:
                while actionAvailable == False:
                    action = random.randint(0, size - 1)
                    actionAvailable = env.checkIfMoveAvailable(action)
                    
            env.step(player + 1, action)
            all_frames.append(env.returnTable().copy())
            winner = env.checkWinner()

            if winner == 1:
                reward = 1
                victories += 1
                
            if winner > 1:
                reward = -1
                penalties += 1
                
            if (player + 1) == 1:
                old_value = q_table[statePosition][action]
                oldState = statePosition

                states, new, statePosition = checkStateIsOnListOrAdd(states, env.returnTable())
                if new == False:
                    next_max = np.max(q_table[statePosition])
                else:
                    q_table.append(np.zeros(size))
                    next_max = 0
            
                new_value = (1 - learning_rate) * old_value + learning_rate * (reward + gamma * next_max)
                q_table[oldState][action] = new_value
                
            epochs += 1

            if winner == -1:
                break
                
print('Total number of moves ' + str(epochs))
print('Total number of victories ' + str(victories))
print('Total number of looses ' + str(penalties))

# With this method we can see the games were played before

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        render(frame)
        sleep(.1)
        
print_frames(all_frames)